## TP Data Cleaning (1) : Handling Missing Values

## Bienvenue aux travaux pratiques de Nettoyage des donn√©es.

Le nettoyage des donn√©es est une partie essentielle de la science des donn√©es, mais cela peut √™tre profond√©ment frustrant. Pourquoi certains de vos champs de texte sont-ils brouill√©s ? Que devez-vous faire √† propos de ces valeurs manquantes ? Pourquoi vos dates ne sont-elles pas correctement format√©es ? Comment pouvez-vous nettoyer rapidement des entr√©es de donn√©es incoh√©rentes ? Dans ces TPs, vous apprendrez pourquoi vous rencontrez ces probl√®mes et, plus important encore, comment les r√©soudre !

Dans ces TPs, vous apprendrez comment aborder certains des probl√®mes de nettoyage de donn√©es les plus courants afin de pouvoir r√©ellement analyser vos donn√©es plus rapidement. Vous travaillerez √† travers cinq exercices pratiques avec des donn√©es r√©elles et d√©sordonn√©es et r√©pondrez √† certaines de vos questions les plus fr√©quemment pos√©es sur le nettoyage des donn√©es.

Dans ce Notebook, nous verrons comment traiter les valeurs manquantes.

## Jeter un premier coup d'≈ìil aux donn√©es
La premi√®re chose que nous devrons faire est de charger les biblioth√®ques et l'ensemble de donn√©es que nous utiliserons.

√Ä titre de d√©monstration, nous utiliserons un ensemble de donn√©es d'√©v√©nements survenus lors de matchs de football. Dans le prochain exercice, vous appliquerez vos nouvelles comp√©tences √† un ensemble de donn√©es de permis de construire.

### √Ä propos de l'ensemble de donn√©es
L'ensemble de donn√©es comporte 407 689 lignes et 100 colonnes. Chaque match est d√©taill√© en fournissant des informations sur : la situation du match, les joueurs impliqu√©s, les r√©sultats et des mesures avanc√©es telles que les valeurs de points attendus et de probabilit√© de victoire.

In [2]:
# modules we'll use
import pandas as pd
import numpy as np

# read in all our data
even_data = pd.read_csv("../chemin de l'ensemble de donn√©es/.csv")

# set seed for reproducibility
np.random.seed(0) 

FileNotFoundError: [Errno 2] No such file or directory: "../chemin de l'ensemble de donn√©es/.csv"

La premi√®re chose √† faire lorsque vous obtenez un nouvel ensemble de donn√©es est d'y jeter un coup d'≈ìil. Cela vous permet de v√©rifier s'il a √©t√© correctement lu et de vous donner une id√©e de ce qui se passe avec les donn√©es. Dans ce cas, voyons s'il y a des valeurs manquantes, qui seront repr√©sent√©es par `NaN` ou `None`.

In [ ]:
# look at the first five rows of the even_data file. 
# I can see a handful of missing data already!
even_data.head()

Oui, il semble qu'il y ait quelques valeurs manquantes.

### Combien de points de donn√©es manquants avons-nous ?

Maintenant que nous savons que nous avons effectivement des valeurs manquantes, voyons combien nous en avons dans chaque colonne.

In [ ]:
# get the number of missing data points per column
missing_values_count = even_data.isnull().sum()

# look at the # of missing points in the first ten columns
missing_values_count[0:10]

Cela semble √™tre beaucoup ! Il pourrait √™tre utile de voir quel pourcentage des valeurs de notre ensemble de donn√©es √©taient manquantes pour avoir une meilleure id√©e de l'ampleur de ce probl√®me :

In [ ]:
# how many total missing values do we have?
total_cells = np.product(even_data.shape)
total_missing = missing_values_count.sum()

# percent of data that is missing
percent_missing = (total_missing/total_cells) * 100
print(percent_missing)

Waouh, pr√®s d'un quart des cellules de cet ensemble de donn√©es sont vides ! Dans l'√©tape suivante, nous allons examiner de plus pr√®s certaines des colonnes avec des valeurs manquantes et essayer de comprendre ce qui pourrait se passer avec elles.

## Comprendre pourquoi les donn√©es sont manquantes

C'est √† ce moment que nous entrons dans la partie de la science des donn√©es que j'aime appeler "intuition des donn√©es", c'est-√†-dire "regarder vraiment vos donn√©es et essayer de comprendre pourquoi elles sont comme elles le sont et comment cela affectera votre analyse". Cela peut √™tre une partie frustrante de la science des donn√©es, surtout si vous √™tes nouveau dans le domaine et que vous n'avez pas beaucoup d'exp√©rience. Pour traiter les valeurs manquantes, vous devrez utiliser votre intuition pour comprendre pourquoi la valeur est manquante. L'une des questions les plus importantes que vous pouvez vous poser pour aider √† comprendre cela est la suivante :

> **Cette valeur manque-t-elle parce qu'elle n'a pas √©t√© enregistr√©e ou parce qu'elle n'existe pas ?**

Si une valeur manque parce qu'elle n'existe pas (comme la taille du plus vieux enfant de quelqu'un qui n'a pas d'enfants), il n'a pas de sens d'essayer de deviner ce qu'elle pourrait √™tre. Vous voudrez probablement conserver ces valeurs en tant que `NaN`. D'autre part, si une valeur manque parce qu'elle n'a pas √©t√© enregistr√©e, alors vous pouvez essayer de deviner ce qu'elle aurait pu √™tre en fonction des autres valeurs dans cette colonne et ligne. Cela s'appelle **l'imputation**, et nous allons apprendre √† le faire ensuite ! :)

Prenons un exemple. En examinant le nombre de valeurs manquantes dans le dataframe `even_data`, je remarque que la colonne "TimesSec" a beaucoup de valeurs manquantes :

In [ ]:
# look at the # of missing points in the first ten columns
missing_values_count[0:10]

En regardant le jeu de donn√©es `even_play2009to2016`, je peux voir que cette colonne contient des informations sur le nombre de secondes restantes dans le match lorsque la jeu a √©t√© effectu√©. Cela signifie que ces valeurs sont probablement manquantes parce qu'elles n'ont pas √©t√© enregistr√©es, plut√¥t que parce qu'elles n'existent pas. Il serait donc logique pour nous d'essayer de deviner ce qu'elles devraient √™tre plut√¥t que de les laisser simplement en tant que NA.

D'autre part, il y a d'autres champs, comme "PenalizedTeam", qui ont √©galement beaucoup de valeurs manquantes. Dans ce cas, cependant, le champ est manquant car s'il n'y avait pas de p√©nalit√©, il n'a pas de sens de dire *quelle* √©quipe a √©t√© p√©nalis√©e. Pour cette colonne, il serait plus logique de laisser vide ou d'ajouter une troisi√®me valeur comme "neither" et l'utiliser pour remplacer les NA.

> **Astuce :** C'est un excellent endroit pour lire la documentation du jeu de donn√©es si vous ne l'avez pas d√©j√† fait ! Si vous travaillez avec un jeu de donn√©es que vous avez obtenu d'une autre personne, vous pouvez √©galement essayer de les contacter pour obtenir plus d'informations.

Si vous effectuez une analyse de donn√©es tr√®s m√©ticuleuse, c'est √† ce moment-l√† que vous examinerez chaque colonne individuellement pour d√©terminer la meilleure strat√©gie pour remplir ces valeurs manquantes. Pour le reste de ce cahier, nous couvrirons quelques techniques "rapides et sales" qui peuvent vous aider avec les valeurs manquantes, mais qui finiront probablement par supprimer certaines informations utiles ou ajouter du bruit √† vos donn√©es.

# Supprimer les valeurs manquantes

Si vous √™tes press√© ou n'avez pas de raison de savoir pourquoi vos valeurs sont manquantes, une option que vous avez est de simplement supprimer toutes les lignes ou colonnes contenant des valeurs manquantes. (Remarque : Je ne recommande g√©n√©ralement pas cette approche pour les projets importants ! Il vaut g√©n√©ralement la peine de prendre le temps de passer en revue vos donn√©es et d'examiner vraiment toutes les colonnes avec des valeurs manquantes une par une pour vraiment conna√Ætre votre jeu de donn√©es.)

Si vous √™tes s√ªr de vouloir supprimer les lignes avec des valeurs manquantes, pandas dispose d'une fonction pratique, `dropna()`, pour vous aider √† le faire. Essayons cela sur notre ensemble de donn√©es d'√©v√©nements !

In [1]:
# remove all the rows that contain a missing value
even_data.dropna()

NameError: name 'even_data' is not defined

il semble que cela ait supprim√© toutes nos donn√©es ! üò± C'est parce que chaque ligne de notre jeu de donn√©es avait au moins une valeur manquante. Nous pourrions avoir plus de chance en supprimant toutes les colonnes qui ont au moins une valeur manquante √† la place.

In [ ]:
# remove all columns with at least one missing value
columns_with_na_dropped = even_data.dropna(axis=1)
columns_with_na_dropped.head()

In [ ]:
# just how much data did we lose?
print("Columns in original dataset: %d \n" % even_data.shape[1])
print("Columns with na's dropped: %d" % columns_with_na_dropped.shape[1])

Nous avons perdu pas mal de donn√©es, mais √† ce stade, nous avons r√©ussi √† supprimer tous les `NaN` de nos donn√©es.

# Remplir automatiquement les valeurs manquantes

Une autre option est d'essayer de remplir les valeurs manquantes. Pour cette prochaine partie, je vais obtenir une petite sous-section des donn√©es de la even afin qu'elles s'impriment correctement.

In [ ]:
# get a small subset of the even dataset
subset_even_data = even_data.loc[:, 'EPA':'Season'].head()
subset_even_data

Nous pouvons utiliser la fonction fillna() de Pandas pour remplir les valeurs manquantes dans un dataframe. Une option que nous avons est de sp√©cifier par quoi nous voulons remplacer les valeurs NaN. Ici, je dis que je voudrais remplacer toutes les valeurs NaN par 0.

In [ ]:
# replace all NA's with 0
subset_even_data.fillna(0)

Je pourrais aussi √™tre un peu plus astucieux et remplacer les valeurs manquantes par la valeur qui vient imm√©diatement apr√®s dans la m√™me colonne. (Cela a beaucoup de sens pour les ensembles de donn√©es o√π les observations ont un certain ordre logique.)

In [ ]:
# replace all NA's the value that comes directly after it in the same column, 
# then replace all the remaining na's with 0
subset_even_data.fillna(method='bfill', axis=0).fillna(0)

# Exercice

√âcrivez votre propre code pour g√©rer les valeurs manquantes dans un ensemble de donn√©es de permis de construire.


1) Jetez un premier coup d'≈ìil aux donn√©es
Ex√©cutez la cellule de code suivante pour charger les biblioth√®ques et l'ensemble de donn√©es que vous utiliserez pour terminer l'exercice.

In [ ]:
# modules we'll use


# read in all our data


# set seed for reproducibility

Affichez les cinq premi√®res lignes du DataFrame sf_permits.

Le jeu de donn√©es contient-il des valeurs manquantes ? 

2) Combien de points de donn√©es manquants avons-nous ?
Quel pourcentage des valeurs dans l'ensemble de donn√©es sont manquantes ? Votre r√©ponse doit √™tre un nombre entre 0 et 100. (Si 1/4 des valeurs dans l'ensemble de donn√©es sont manquantes, la r√©ponse est 25.)

In [ ]:
percent_missing =

3) Comprenez pourquoi les donn√©es sont manquantes
Examinez les colonnes "Street Number Suffix" et "Zipcode" de l'ensemble de donn√©es des permis de construire. Les deux contiennent des valeurs manquantes.

Lesquelles, le cas √©ch√©ant, manquent parce qu'elles n'existent pas ?
Lesquelles, le cas √©ch√©ant, manquent parce qu'elles n'ont pas √©t√© enregistr√©es ?

4. Supprimer les valeurs manquantes : lignes
Si vous supprimez toutes les lignes de sf_permits contenant des valeurs manquantes, combien de lignes restent ?


5) Supprimer les valeurs manquantes : colonnes
Maintenant, essayez de supprimer toutes les colonnes avec des valeurs vides.

Cr√©ez un nouveau DataFrame appel√© sf_permits_with_na_dropped qui a toutes les colonnes avec des valeurs vides supprim√©es.
Combien de colonnes ont √©t√© supprim√©es du DataFrame sf_permits original ? Utilisez ce nombre pour d√©finir la valeur de la variable dropped_columns ci-dessous.

In [ ]:
sf_permits_with_na_dropped =

dropped_columns =

6) Remplir automatiquement les valeurs manquantes
Essayez de remplacer tous les NaN dans les donn√©es sf_permits par celui qui vient imm√©diatement apr√®s, puis remplacez tous les NaN restants par 0. D√©finissez le r√©sultat sur un nouveau DataFrame sf_permits_with_na_imputed.

In [ ]:
sf_permits_with_na_imputed =